In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
df = pd.read_csv('train.csv')  
testdf = pd.read_csv('test.csv')  # the data without class for submission

# Bayesian logistic regression v1
After a few uploads that scored 0 (in the begining I thought that they didn't gain an hypothetic threshold) due to submitting predictions with floats rather than ints, the final upload scored 74%

In [ ]:
# Let's start with some of the cleanings performed with JeffD
d0 = df.copy()
t0 = testdf.copy()

for dataset in (d0, t0):
    # Let's assign unknown ages -1 (to see what happens)
    unknown_ages = dataset[dataset.Age.isna()].index
    dataset.loc[unknown_ages, 'Age'] = -1
    
    # Let's assign nan fares to -1
    dataset.Fare.fillna(-.5, inplace=True)
    
    # Let's make sex a number
    idx = dataset[dataset.Sex == 'female'].index
    dataset.loc[:, 'Sex'] = 0
    dataset.loc[idx, 'Sex'] = 1
    
    
    # Simplify cabin names
    dataset.loc[:, 'Cabin'] = dataset.Cabin.fillna('N')
    dataset.loc[:, 'Cabin'] = dataset.Cabin.apply(lambda x: x[0])
    
    
    # Get the name titles
    d1 = dataset.Name.apply(lambda x: x.split(',')[1].split('.')[0])
    dataset['Title'] = d1.str.replace(' ', '')

    # A couple of irregular ones
    d1 = dataset[dataset.Title.str.contains('Jonkheer')]
    d2 = dataset[dataset.Title.str.contains('Countess')]
    dataset.loc[d1.index, 'Title'] = 'Mr'
    dataset.loc[d2.index, 'Title'] = 'Mrs'  # In her Age group are majority
    
    # Finally, drop some columns
    dataset.drop(columns=['Ticket', 'Embarked', 'Name', 'PassengerId'], inplace=True)

# Let's deal with unknown ages: assign the mean age taking into account the survival status.
# First calculate the mean of the known ages, and then input it into the unknowns
# This didn't work, though, since test set has no survival data and nan ages can't be imputated. 
# age_nan = d0.Age.isna()
# age_dead_mean = d0[~(age_nan) & (d0.Survived == 0)].mean()
# age_alive_mean = d0[~(age_nan) & (d0.Survived == 1)].mean()
# unknown_dead = d0[(age_nan) & (d0.Survived == 0)]
# unknown_alive = d0[(age_nan) & (d0.Survived == 1)]
# d0.loc[unknown_alive.index, 'Age'] = age_alive_mean
# d0.loc[unknown_dead.index, 'Age'] = age_dead_mean


# Encode cabin data:
# Get a feel of surviving chances by type of cabin
cabins = d0.pivot_table(index='Cabin', columns='Survived', values='Sex', aggfunc='count').fillna(0)
cabins['ratio'] = cabins[1] / cabins[0]
# And apply to the data
d0 = pd.merge(d0, cabins.ratio, left_on='Cabin', right_index=True, how='left')
d0.drop(columns=['Cabin',], inplace=True)
d0.rename(columns={'ratio': 'Cabin'}, inplace=True)
t0 = pd.merge(t0, cabins.ratio, left_on='Cabin', right_index=True, how='left')
t0.drop(columns=['Cabin',], inplace=True)
t0.rename(columns={'ratio': 'Cabin'}, inplace=True)

# Encode title TODO

In [ ]:
# Compute a correlation matrix
corr = d0.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
                linewidths=.5, cbar_kws={"shrink": .5}, ax=ax);

 We see here that:
 * Fare is highly and positively correlated with Surviving chances (.26 out of 1) 
 * Otherwise, Pclass is negatively correlated (-.35 out of 1)

### Split Train/test
Split the original train data into subsets of train/test

In [ ]:
train, test = train_test_split(d0, test_size=.3)

In [ ]:
with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula(
        'Survived ~ Sex + Pclass + Fare + Age + Cabin', 
        train[['Survived', 'Sex', 'Pclass', 'Fare', 'Age', 'Cabin']],
        family=pm.glm.families.Binomial())
    trace = pm.sample(3000, tune=3500, init='adapt_diag')

# Get a simple bayesian point estimation
sex_estimate = trace['Sex'].mean()
pclass_estimate = trace['Pclass'].mean()
fare_estimate = trace['Fare'].mean()
age_estimate = trace['Age'].mean()
cabin_estimate = trace['Cabin'].mean()
icept_estimate = trace['Intercept'].mean()

In [ ]:
pm.plot_trace(trace);

In [ ]:
from sklearn.metrics import accuracy_score

test = test.copy()
test['lm'] = (
    sex_estimate * test.Sex +
    pclass_estimate * test.Pclass +
    fare_estimate * test.Fare +
    age_estimate * test.Age +
    cabin_estimate * test.Cabin +
    icept_estimate)

test['logit'] = 1 / (1 + np.exp(-test.lm))

# test['y_hat'] = test.logit.round().astype(int)
test['y_hat'] = 0
idx = test[test.logit > .5].index
test.loc[idx, 'y_hat'] = 1
test['loss'] = (test.Survived - test.y_hat)**2
1 - test.loss.mean()
accuracy_score(test.Survived, test.y_hat)

In [ ]:
lm = (sex_estimate * t0.Sex +
      pclass_estimate * t0.Pclass +
      fare_estimate * t0.Fare +
      age_estimate * t0.Age +
      cabin_estimate * t0.Cabin +
      icept_estimate)

logit = 1 / (1 + np.exp(-lm))
y_hat = np.zeros(logit.size)
y_hat[logit >= .5] = 1

pd.Series(
    index=testdf.PassengerId, data=y_hat.astype(int), name='Survived').to_csv(
    '04-Bayesian-logistic-regression.csv')

In [ ]:
y_hat.astype(int)